The goal of this script is to merge the description from the fasta headers with the orthogroups sequences by using 'Gene' key found in both

In [1]:
import pandas as pd 
import numpy as np
import os
import sys
import re

In [3]:
path = '/home/jake/Downloads/Salmo_Project/'
Orthogroups = f'{path}Orthogroups/'
Orthogroups = pd.read_csv(f'{Orthogroups}Orthogroups.tsv', sep = '\t', index_col=0)
file = '/home/jake/Downloads/HSP_Project/headerz/headerz.txt'
headers = pd.read_csv(file, sep='\t', header=None, names=['header'])
# extract gene up until the first space
headers['gene'] = headers['header'].str.extract(r'gene:(.*?)\s').fillna('')

# extract description
headers['description'] = headers['header'].str.extract(r'description:(.*)\[').fillna('')
#Drop the rows that have nothing in the description column
headers = headers[headers['description'] != '']
headers = headers[['gene', 'description']]
headers_df = headers[['gene', 'description']].astype(str)

The code below produces a df('final_ortho_df') that contains all the orthogroups and their sequences per speceies with an added description column derived from the fasta headers.

In [4]:
print(f'There are {len(Orthogroups)} orthogroups in the Orthogroups.tsv file')
print(f'There are {len(headers_df)} genes in the headers file')
# Load the data into a DataFrame
# Calculate the total number of columns
total_cols = 84
# Calculate the number of columns with missing values (null values) for each row
null_cols = Orthogroups.isna().sum(axis=1)
# Filter out rows that have more than 5% of the columns with missing values
filtered_df = Orthogroups[null_cols / total_cols <= 0.05]
print(f'There are {len(filtered_df)} orthogroups in the filtered_df file')
#Reset the index for both dataframes
filtered_df = filtered_df.reset_index()
headers_df = headers_df.reset_index()
#Display the first 5 rows of the filtered DataFrame
#make a new df that has two columns, the Orthogroup column and genes column that merge all of the columns in filtered_df into one column
new_df = pd.DataFrame(columns=['Orthogroup', 'gene'])
new_df['Orthogroup'] = filtered_df['Orthogroup']
new_df['gene'] = filtered_df.iloc[:,1:].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
#Pivot the new_df so each string in the genes column is a seperate row
new_df = new_df.set_index('Orthogroup')['gene'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='gene')
# merge the two dataframes on the gene column
final_df = pd.merge(headers_df, new_df, on='gene', how='left')
#Drop rows where Orthogroup is NaN
#Drop the index column
final_df = final_df.dropna(subset=['Orthogroup'])
#Group by Orthogroup and concatenate the description , and gene columns seperated by a '/'
final_df = final_df.groupby('Orthogroup')['description', 'gene'].agg(lambda x: '/'.join(x)).reset_index()
#add the description column to the filtered_df based on orthogroup
temp_df = pd.merge(filtered_df, final_df, on='Orthogroup', how='left')
final_ortho_df = temp_df.drop(columns=['gene'])
#drop the rows with where description column is NaN

final_ortho_df = final_ortho_df.dropna(subset=['description'])


There are 27791 orthogroups in the Orthogroups.tsv file
There are 3753907 genes in the headers file
There are 23182 orthogroups in the filtered_df file


/tmp/ipykernel_25728/4112835504.py:27: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  final_df = final_df.groupby('Orthogroup')['description', 'gene'].agg(lambda x: '/'.join(x)).reset_index()


In [5]:
#THis cell gets the heat shock orhtogroups
_ = final_ortho_df[final_ortho_df['description'].str.contains('heat|hsp|HSP|shps|HEAT|shock', case=False)]

def count_non_na_columns(df):
    df['%conserved'] = df.iloc[:, 1:85].count(axis=1)
    return df
_ = count_non_na_columns(_)

hsps = _[['Orthogroup', 'description', '%conserved']]
hsps.to_csv(f'{path}hsps.csv', sep=',', index=False)

/tmp/ipykernel_25728/249907178.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['%conserved'] = df.iloc[:, 1:85].count(axis=1)


In [6]:
#THis cell gets the non heat shock orthhogroups
#filter out the rows that have the word heat in the description column
not__= final_ortho_df[~final_ortho_df['description'].str.contains('heat|hsp|HSP|shps|HEAT|shock|factor|domain|transcription', case=False)]

def count_non_na_columns(df):
    df['%conserved'] = df.iloc[:, 1:85].count(axis=1)
    return df
not_hsps = count_non_na_columns(not__)
not_hsps = not_hsps[['Orthogroup', 'description', '%conserved']]
#Ranomly sample 70 rows from the not_hsps dataframe
not_hsps_sample = not_hsps.sample(n=70, random_state=1)
not_hsps_sample.to_csv(f'{path}not_hsps.csv', sep=',', index=False)
display(not_hsps_sample)

/tmp/ipykernel_25728/1140220477.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['%conserved'] = df.iloc[:, 1:85].count(axis=1)


,Orthogroup,description,%conserved
2444,OG0002545,interleukin 6 signal transducer /interleukin 6...,8
12019,OG0012178,"scavenger receptor class B, member 2b /scaveng...",8
10105,OG0010243,mitochondrial ribosomal protein S7 /mitochondr...,8
1273,OG0001354,optineurin /optineurin /optineurin /optineurin...,8
10607,OG0010746,cell division cycle 14Ab /cell division cycle ...,7
...,...,...,...
12735,OG0012908,si:dkeyp-20g2.1 /si:dkeyp-20g2.1 /coiled-coil ...,8
894,OG0000966,retinoid isomerohydrolase RPE65 a /retinoid is...,8
9332,OG0009467,MORN repeat containing 4 /MORN repeat containi...,7
14007,OG0014200,si:ch1073-143l10.2 /ATG16 autophagy related 16...,8
